<a href="https://colab.research.google.com/github/kunalavghade/Ai/blob/main/Week_12_%E2%80%94_Recurrent_Neural_Networks_%26_Backpropagation_Through_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 12 — Recurrent Neural Networks & Backpropagation Through Time

## Goal

Understand how neural networks process sequences and learn temporal dependencies.

By the end of this week, you should:
- Implement a vanilla RNN cell from scratch
- Understand hidden state dynamics
- Implement Backpropagation Through Time (BPTT)
- Predict simple sequences
- Observe exploding/vanishing gradients

This week introduces time into neural networks.

---

# 1. Why Sequential Models?

Unlike images:
- Text
- Time series
- Speech
- Sensor signals

Have temporal dependency.

Order matters.

Example:
"The movie was not good"

Meaning depends on sequence.

---

# 2. Vanilla RNN Architecture

At time step t:

h_t = tanh(W_x x_t + W_h h_{t-1} + b)

y_t = W_y h_t + c

Where:
- x_t = input at time t
- h_t = hidden state
- W_x = input weights
- W_h = recurrent weights
- W_y = output weights

The hidden state stores memory.

---

# 3. Key Idea: Parameter Sharing Across Time

Same weights:
- Used at every time step

This is like:
- Deep network unrolled over time

---

# 4. Unrolling Through Time

Sequence length T:

x1 → h1 → h2 → h3 → ... → hT

This is equivalent to:
- A deep network of depth T
- With shared weights

---

# 5. Backpropagation Through Time (BPTT)

To compute gradients:
- Unroll network
- Apply chain rule across time

Gradient at time t depends on:
- All future losses

This leads to:

∂L/∂W_h = Σ over time

---

# 6. Vanishing & Exploding Gradients in RNNs

Repeated multiplication:

∂h_t/∂h_{t-1} = W_h * derivative(tanh)

If eigenvalues < 1 → vanishing  
If eigenvalues > 1 → exploding  

This problem is worse than in deep CNNs.

---

# 7. Truncated BPTT

Instead of backprop through entire sequence:
- Backprop only k steps

Benefits:
- Reduces computation
- Stabilizes training

Tradeoff:
- May miss long-term dependencies

---

# 8. Sequence Prediction Setup

Two common tasks:

1. Many-to-one:
   - Input sequence → single output

2. Many-to-many:
   - Input sequence → output at each time step

---

# Coding Exercises

## Question 1: Implement RNN Cell

Implement forward step:

h_t = tanh(W_x x_t + W_h h_prev + b)

Return:
- h_t
- cache for backward

---

## Question 2: Full Forward Pass

Given sequence of length T:
- Loop over time
- Store hidden states

Test with:
- Random data

---

## Question 3: Backprop Through Time

Implement:
- Gradients w.r.t W_x
- Gradients w.r.t W_h
- Gradients w.r.t b

Accumulate gradients over time.

Verify with numerical gradient checking.

---

## Question 4: Predict Number Sequence

Train RNN on:
- Input: [1,2,3,4]
- Target: [2,3,4,5]

Or simple sine wave.

Observe:
- Learning behavior
- Gradient norms

---

## Question 5: Character-Level Toy Dataset

Create small dataset:
- "hello world"
Train RNN to:
- Predict next character

Observe:
- Generated sequences
- Training stability

---

## Question 6: Implement Truncated BPTT

Modify training:
- Backprop only k steps (e.g., k=5)

Compare:
- Full BPTT vs truncated
- Stability
- Speed

---

# Conceptual Questions

1. Why does unrolling create a deep network?
2. Why do gradients vanish faster in RNNs?
3. Why does tanh contribute to vanishing gradients?
4. Why is truncated BPTT useful?
5. Why is sequence length critical?

---

# Outcome of Week 12

After this week, you should:
- Understand RNN mechanics deeply
- Implement BPTT from scratch
- See exploding gradients firsthand
- Feel the limitations of vanilla RNNs

In [1]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

🔹 PART 1 — RNN Cell (Single Time Step)

We implement:
  H𝑡 = tanh(𝑊𝑥X𝑡 +𝑊ℎH𝑡−1+𝑏)
	​

**✅ Question 1 — RNN Cell Forward**

In [24]:
class RNNCell:
  def __init__(self, input_dim, hidden_dim):
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim

    self.Wx = np.random.randn(hidden_dim, input_dim) * 0.1
    self.Wh = np.random.randn(hidden_dim, hidden_dim) * 0.1
    self.b = np.zeros((hidden_dim, 1))

  def forward(self, x_t, h_prev):
    z = self.Wx @ x_t + self.Wh @ h_prev + self.b
    h_t = np.tanh(z)
    cache = (x_t, h_prev, z, h_t)
    return h_t, cache

Test:

In [25]:
cell = RNNCell(3,5)

x = np.random.randn(3,1)
h_prev = np.random.randn(5,1)

h_t, cache = cell.forward(x, h_prev)

print(h_t.shape)

(5, 1)



🔹 **PART 2 — Full Forward Through Time**

✅ Question 2 — Sequence Forward

In [26]:
def rnn_forward(cell, X_seq):
  h_prev = np.zeros((cell.hidden_dim, 1))

  caches, ha_states = [], []
  for x_t in X_seq:
    h_prev, cache = cell.forward(x_t, h_prev)
    caches.append(cache)
    ha_states.append(h_prev)
  return ha_states, caches

Test:

In [27]:
X_seq = [np.random.randn(3, 1) for _ in range(4)]
h_states, caches = rnn_forward(cell, X_seq)

print(len(h_states))
print(h_states[0].shape)

4
(5, 1)


**PART 3 — Backpropagation Through Time (BPTT)**

✅ **Question 3 — RNN Backward (BPTT)**

We compute gradients for:
- Wx
- Wh
- b

Key equation:

$\frac{\partial L}{\partial h_t} =
\frac{\partial L_t}{\partial h_t} +
W_h^{T} \frac{\partial L}{\partial h_{t+1}}$

## Meaning (Quick Intuition)

- $L$ → total loss  

- $L_t$ → loss at time step $t$  

- $h_t$ → hidden state at time $t$  

- $W_h$ → recurrent weight matrix  

---

### What This Equation Tells Us

It shows that the gradient at time $t$ depends on:

1. **The direct loss at time $t$**
   
   $$
   \frac{\partial L_t}{\partial h_t}
   $$

2. **The gradient flowing back from time $t+1$**
   
   $$
   W_h^T \frac{\partial L}{\partial h_{t+1}}
   $$

---

### Intuition in Words

When computing gradients in an RNN:

- The hidden state $h_t$ affects:
  - The loss at the current time step.
  - Future hidden states $h_{t+1}, h_{t+2}, \dots$

- So its gradient must accumulate:
  - Immediate contribution.
  - Future contribution passed backward through the recurrent weights.

This recursive dependency is what leads to vanishing or exploding gradients in long sequences.

This is the recursive gradient accumulation.

In [33]:
def rnn_backward(cell, dh_list, caches):
  """
    dh_list: list of dL/dh_t for each time step
  """
  dWx = np.zeros_like(cell.Wx)
  dWh = np.zeros_like(cell.Wh)
  db = np.zeros_like(cell.b)

  dh_next = np.zeros((cell.hidden_dim, 1))
  for t in reversed(range(len(caches))):
    x_t, h_prev, z, h_t = caches[t]
    dh = dh_list[t] + dh_next

    dz = dh * (1 - h_t**2) # tanh derivative

    dWx += dz @ x_t.T
    dWh += dz @ h_prev.T
    db += dz

    dh_next = cell.Wh.T @ dz

  return dWx, dWh, db

Numerical Gradient Check

In [34]:
def numerical_gradient_Wx(cell, X_seq, eps=1e-5):
  grad = np.zeros_like(cell.Wx)

  for i in range(cell.Wx.shape[0]):
    for j in range(cell.Wx.shape[1]):
      original = cell.Wx[i, j]

      cell.Wx[i, j] = original + eps
      h_plus, _ = rnn_forward(cell, X_seq)
      loss_plus = sum(np.sum(h) for h in h_plus)


      cell.Wx[i, j] = original - eps
      h_minus, _ = rnn_forward(cell, X_seq)
      loss_minus = sum(np.sum(h) for h in h_minus)

      grad[i, j] = (loss_plus - loss_minus) / (2 * eps)
      cell.Wx[i, j] = original
  return grad

Test:

In [35]:
cell = RNNCell(2,3)
X_seq = [np.random.randn(2,1) for _ in range(3)]

h_states, caches = rnn_forward(cell, X_seq)
dh_list = [np.ones_like(h) for h in h_states]

dWx, dWh, db = rnn_backward(cell, dh_list, caches)
dWx_num = numerical_gradient_Wx(cell, X_seq)

print("Difference:", np.linalg.norm(dWx - dWx_num))

Difference: 2.836222449079511e-10


PART 4 — Train RNN on Number Sequence

Task:

Input: [1,2,3,4...100]

Target: [2,3,4,5...100]

In [36]:
def generate_sequence(start, length):
  X, Y = [],[]
  for i in range(length):
    X.append(np.array([[start + i]]))
    Y.append(np.array([[start + i + 1]]))
  return X, Y


Add Output Layer

In [37]:
class RNNModel:
  def __init__(self):
    self.cell = RNNCell(1, 10)
    self.Wy = np.random.randn(1, 10) * 0.1
    self.by = np.zeros((1,1))

  def forward(self, X_seq):
    h_states, caches = rnn_forward(self.cell, X_seq)
    outputs = []
    for h in h_states:
      output = self.Wy @ h + self.by
      outputs.append(output)
    return outputs, h_states, caches

Training Loop

In [85]:
model = RNNModel()
lr = 0.001
X_seq, Y_seq = generate_sequence(1, 10)
print(X_seq)
print(Y_seq)

losses = []
logging = True
epochs = [ i for i in range(5000)]

grad_clip = 1.0 # Define gradient clipping threshold

for epoch in epochs:
  outputs, h_states, caches = model.forward(X_seq)
  loss, dh_list = 0, []

  for y_pred, y_true, h in zip(outputs, Y_seq, h_states):
    loss += np.sum((y_pred - y_true)**2)
    dy = 2 * (y_pred - y_true)
    dh = model.Wy.T @ dy
    dh_list.append(dh)

    # Apply gradient clipping for output layer weights
    model.Wy -= lr * np.clip(dy @ h.T, -grad_clip, grad_clip)
    model.by -= lr * np.clip(dy, -grad_clip, grad_clip)

  dWx, dWh, db = rnn_backward(model.cell, dh_list, caches)

  # Apply gradient clipping for RNN cell weights
  model.cell.Wx -= lr * np.clip(dWx, -grad_clip, grad_clip)
  model.cell.Wh -= lr * np.clip(dWh, -grad_clip, grad_clip)
  model.cell.b -= lr * np.clip(db, -grad_clip, grad_clip)

  losses.append(loss)
  if epoch % 100 == 0 and logging:
    print(f"Epoch {epoch}, Loss: {loss}")


[array([[1]]), array([[2]]), array([[3]]), array([[4]]), array([[5]]), array([[6]]), array([[7]]), array([[8]]), array([[9]]), array([[10]])]
[array([[2]]), array([[3]]), array([[4]]), array([[5]]), array([[6]]), array([[7]]), array([[8]]), array([[9]]), array([[10]]), array([[11]])]
Epoch 0, Loss: 490.7886741409609
Epoch 100, Loss: 29.238136182196502
Epoch 200, Loss: 46.274723643485245
Epoch 300, Loss: 52.30624251792246
Epoch 400, Loss: 54.9779100398943
Epoch 500, Loss: 56.573447268372256
Epoch 600, Loss: 57.48037253660326
Epoch 700, Loss: 58.084664333685154
Epoch 800, Loss: 58.562102678633735
Epoch 900, Loss: 58.957611453945276
Epoch 1000, Loss: 59.294006780229545
Epoch 1100, Loss: 59.58584660131686
Epoch 1200, Loss: 59.842985280798274
Epoch 1300, Loss: 60.07238848895922
Epoch 1400, Loss: 60.27915284669405
Epoch 1500, Loss: 60.46711408091531
Epoch 1600, Loss: 60.63922758229825
Epoch 1700, Loss: 60.79781617413503
Epoch 1800, Loss: 60.94473689812552
Epoch 1900, Loss: 61.08149652884167


In [86]:
print("Gradient norm:", np.linalg.norm(dWh))

Gradient norm: 0.015477958438056636


In [91]:
# Make a prediction with the trained model
input_value = np.array([[3]]) # Example input

# The model.forward expects a list of inputs, even for a single time step
outputs, _, _ = model.forward([input_value])

print(f"Input: {input_value[0][0]}")
print(f"Predicted output: {outputs[0][0][0]:.4f}")

Input: 3
Predicted output: 6.0233


PART 5 — Character Level Toy Dataset

In [79]:
text = "hello world"
chars = list(set(text))
char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for ch,i in char_to_ix.items()}

In [81]:
def one_hot(idx, size):
    vec = np.zeros((size,1))
    vec[idx] = 1
    return vec

PART 6 — Truncated BPTT

Modify backward loop:

Instead of:



```
for t in reversed(range(len(caches))):
```

```
for t in reversed(range(max(0, len(caches)-k), len(caches))):
```



In [93]:
def rnn_backward(cell, dh_list, caches):
  """
    dh_list: list of dL/dh_t for each time step
  """
  k = 5
  dWx = np.zeros_like(cell.Wx)
  dWh = np.zeros_like(cell.Wh)
  db = np.zeros_like(cell.b)

  dh_next = np.zeros((cell.hidden_dim, 1))
  for t in reversed(range(max(0, len(caches)-k), len(caches))):
    x_t, h_prev, z, h_t = caches[t]
    dh = dh_list[t] + dh_next

    dz = dh * (1 - h_t**2) # tanh derivative

    dWx += dz @ x_t.T
    dWh += dz @ h_prev.T
    db += dz

    dh_next = cell.Wh.T @ dz

  return dWx, dWh, db